In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']= 100
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [12]:
def in_opponent_base(x,y,opponent):

'''
returns indicator function of whether the player is in the base of the opponent
x,y - coordinates
opponent = {'Radient', 'Dire'} - opponent team

'''
radiant_base_x = 96
radiant_base_y = 100

dire_base_x = 156
dire_base_y = 156

if opponent == 'Radient':
    if x <= radiant_base_x and y <= radiant_base_y:
        return 1
    else:
        return 0
elif opponent == 'Dire':
    if x >= dire_base_x and y >= dire_base_y:
        return 1
    else:
        return 0
else:
    return NaN

  


In [88]:
# load the data and combine with test data to perform some of the trasformations (without standartization) on all data at the same time
train_df = pd.read_csv('train_features.csv', index_col='match_id_hash')
test_df = pd.read_csv('test_features.csv', index_col='match_id_hash')
train_ind = data_train.index
full_df = pd.concat([train_df, test_df])
new_features = pd.DataFrame(index=full_df.index)
full_df.head(2)

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,11,0,0,0,0,...,0.0,0,0,0,0,0.0,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,...,0.0,0,0,0,0,0.0,0,0,0,0


In [89]:
new_feats = [i.replace("1","")  for i in full_df.columns.values if (i.startswith('r1')==1 or i.startswith('d1')==1)]

In [90]:
rem_feat = ['r_hero_id', 'r_health', 'r_max_health','r_level', 'r_max_mana', 'r_x', 'r_y', 'r_teamfight_participation', 'd_hero_id', 'd_health','d_max_health', 'd_max_mana', 'd_level', 'd_x', 'd_y', 'd_teamfight_participation']
for feat in rem_feat:
    new_feats.remove(feat)

Create a feature that is sum of r1 + ... + r5 for all r features, and same for all d features. Note that not all of these make sense, so some should be removed.

In [91]:
for feature in new_feats:
    feat_names = [(feature[0] + '{}_'.format(i) +feature[2:]) for i in range(1,6)]
    new_features[feature] = full_df.loc[:,feat_names].sum(axis=1)

In [92]:
new_features.columns

Index(['r_kills', 'r_deaths', 'r_assists', 'r_denies', 'r_gold', 'r_lh',
       'r_xp', 'r_stuns', 'r_creeps_stacked', 'r_camps_stacked',
       'r_rune_pickups', 'r_firstblood_claimed', 'r_towers_killed',
       'r_roshans_killed', 'r_obs_placed', 'r_sen_placed', 'd_kills',
       'd_deaths', 'd_assists', 'd_denies', 'd_gold', 'd_lh', 'd_xp',
       'd_stuns', 'd_creeps_stacked', 'd_camps_stacked', 'd_rune_pickups',
       'd_firstblood_claimed', 'd_towers_killed', 'd_roshans_killed',
       'd_obs_placed', 'd_sen_placed'],
      dtype='object')

In [93]:
new_features.head()

,r_kills,r_deaths,r_assists,r_denies,r_gold,r_lh,r_xp,r_stuns,r_creeps_stacked,r_camps_stacked,...,d_xp,d_stuns,d_creeps_stacked,d_camps_stacked,d_rune_pickups,d_firstblood_claimed,d_towers_killed,d_roshans_killed,d_obs_placed,d_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,1,0,4,2037,25,1991,0.000000,0,0,...,3084,0.299948,0,0,7,1,0,0,1,0
b9c57c450ce74a2af79c9ce96fac144d,16,4,14,28,18172,127,17566,42.060296,2,1,...,12925,3.532506,3,2,14,1,0,0,5,2
6db558535151ea18ca70a6892197db41,0,0,0,0,880,0,0,0.000000,0,0,...,0,0.000000,0,0,1,0,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,6,2,8,43,13348,131,14663,11.397237,0,0,...,11843,13.663390,6,2,11,0,0,0,3,0
b1b35ff97723d9b7ade1c9c3cf48f770,2,4,3,0,8079,62,9762,15.262940,0,0,...,10806,26.826797,2,1,8,0,0,0,0,0


plan:
    1. aggregate by sum() everything possible
    2. get proportions of health and mana and generale quantiles from them
    3. generate something for level
    4. dummy variables for game_mode
    5. ....scaling will be done in the training part (CV)